In [ ]:
#@title Press to predict { run: "auto", vertical-output: true, display-mode: "form" }
from IPython.utils import io
with io.capture_output() as captured:
  !pip install yfinance
  !pip install pandas_market_calendars
  !pip install trading_calendars
  !pip install pytz
from IPython.display import clear_output
from IPython.display import display,HTML
import ipywidgets as widgets
import os.path
from os import path
from IPython.display import display, clear_output
import pytz
import trading_calendars as tc
import yfinance as yf
import datetime as dt
import numpy as np
from numpy import load
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
from mlxtend.preprocessing import minmax_scaling
from datetime import datetime,timedelta
import pandas_market_calendars as mcal
from time import sleep
import urllib.request
from google.colab import files
with tf.device('/device:GPU:0'):
  clear_output()
  urllib.request.urlretrieve('https://drive.google.com/u/0/uc?id=1BCgC2o2wGfQqmeJjlhSe-EElT0KzzxZa&export=download','modelevaluate0.0019.h5') 
  urllib.request.urlretrieve('https://drive.google.com/u/0/uc?id=1-PdwcdBYsEW_FPpiFEYv8OqdcsmA8Eji&export=download','modelprecentfirst0.0015.h5')
  sleep(3)  
  model = tf.keras.models.load_model('/content/modelprecentfirst0.0015.h5')
  modelevaluate = tf.keras.models.load_model('/content/modelevaluate0.0019.h5')
 
  def evaluatemodel(stocksnames,dictofdata,topstocks):
    dicev={}
    dicp={}
    dicgrade={}
    for sn in stocksnames:
      z1=dictofdata[sn]
      ev=modelevaluate(z1)
      ev=ev.numpy()
      ev=ev.astype(np.float32)
      p = model(z1)
      p = p.numpy()
      p=p.astype(np.float32)
      dicp[sn]=p[0,9]
      dicev[sn]=abs(ev[0,9])
    stocksortev=sorted(dicev, key=dicev.get)
    stocksortp=sorted(dicp, key=dicp.get,reverse=True)
    print(dicp[stocksortp[0]])
    print(dicev[stocksortev[0]])
    grade=0
    for sn in stocksnames:
      if sn in stocksortp:
        grade=0.0
        grade=float(stocksortev.index(sn))*2.5
        grade=grade+float(stocksortp.index(sn))
        dicgrade[sn]=grade
    stocksort=sorted(dicgrade, key=dicgrade.get)

    return stocksort[0:topstocks]

  def createmodeldata(stocksnames,datadic,daysago):
    dicdata={}
    snf=[]
    snf1=[]
    k=-1
    inputnotorder1=datadic['s&p']
    r2 = len(inputnotorder1)- daysago
    normalhelp1 = inputnotorder1[r2-25:r2,3].astype(np.float32)
    mini1 = np.min(normalhelp1)
    maxi1 = np.max(normalhelp1)
    normalhelp1 = (normalhelp1 - mini1) / (maxi1 - mini1)*2-1
    normalhelp1 = np.nan_to_num(normalhelp1, nan=0.5, posinf=1, neginf=0)
    for sn in stocksnames:
      stockname = sn
      k=k+1
      stockssortedin = {};
      stockssortedout = {};
      mam = {}
      inputnotorder = datadic[sn];
      normalhelp = [];
      calcu=[]
      vol = [];
      stockstopresent=np.arange(10).astype(np.float32)
      allthestocks=np.arange(35).astype(np.float32)
      allthetrends=np.arange(35).astype(np.float32)
      calcuall=np.arange(45).astype(np.float32)
      su1=np.arange(25).astype(np.float32)
      su2=np.arange(25).astype(np.float32)
      re1=np.arange(25).astype(np.float32)
      re2=np.arange(25).astype(np.float32)
      trendprediction=np.arange(25).astype(np.float32)
      momentumall=np.arange(35).astype(np.float32)
      momentum=np.arange(25).astype(np.float32)
      ppt=np.arange(25).astype(np.float32)
      r1 = len(inputnotorder) - daysago
      normalhelp = inputnotorder[r1-25:r1, 1:4].astype(np.float32)
      calcu=inputnotorder[r1-35:r1, 1:4].astype(np.float32)
      mini = np.min(normalhelp)
      maxi = np.max(normalhelp)
      mam[0] = [maxi, mini]
      normalhelp = (normalhelp - mini) / (maxi - mini)*2-1
      normalhelp = np.nan_to_num(normalhelp, nan=0.5, posinf=1, neginf=0)
      for q in range(0,25):
        ppt[q]=(calcu[q+9,0]+calcu[q+9,1]+calcu[q+9,2])/3
        su1[q]=((ppt[q]*2)-(calcu[q+9,0]))
        su2[q]=((ppt[q])+(calcu[q+9,1])-(calcu[q+9,0]))
        re1[q]=((ppt[q]*2)-(calcu[q+9,1]))
        re2[q]=((ppt[q])+(calcu[q+9,0])-(calcu[q+9,1]))
      for q in range(0,25):
        trendprediction[q]=((calcu[q+10,2] / calcu[q+9,2])-1)
      for q in range(0,25):
        momentum[q]=((calcu[q+10,2])/(calcu[q,2])-1)
 
 
      su1=(su1 - mini) / (maxi - mini)*2-1
      su2=(su2 - mini) / (maxi - mini)*2-1
      re1=(re1 - mini) / (maxi - mini)*2-1
      re2=(re2 - mini) / (maxi - mini)*2-1
 
 
      
      normalhelp = np.column_stack((normalhelp, trendprediction))
      normalhelp = np.column_stack((normalhelp, momentum))
      normalhelp = np.column_stack((normalhelp, su1))
      normalhelp = np.column_stack((normalhelp, su2))
      normalhelp = np.column_stack((normalhelp, re1))
      normalhelp = np.column_stack((normalhelp, re2))
      normalhelp = np.column_stack((normalhelp, normalhelp1))
 
 
      stockssortedin[0] = normalhelp[0:25, :]
      x = list(stockssortedin.values())
      mm = list(mam.values())
      z2 = np.expand_dims(x[0], axis=0)
      z2=z2.astype(np.float32)
      dicdata[sn]=z2
    return dicdata
 
  def presentandsave(stocksnames,datadic):
    lo=0
    la=len(stocksnames)
    lod=widgets.FloatProgress(value=lo,min=0,max=la,description='Loading:',bar_style='info',style={'bar_color': '#ffff00'},orientation='horizontal')
    
    k=-1
    snf=[]
    predictionstocks={}
    trends={};
    su1alld={}
    su2alld={}
    re1alld={}
    re2alld={}
    dicdata={}
    momentumd={}
    inputnotorder1 = datadic['s&p']
    r2 = len(inputnotorder1)
    normalhelp1 = inputnotorder1[r2-25:r2,3].astype(np.float32)
    mini1 = np.min(normalhelp1)
    maxi1 = np.max(normalhelp1)
    normalhelp1 = (normalhelp1 - mini1) / (maxi1 - mini1)*2-1
    normalhelp1 = np.nan_to_num(normalhelp1, nan=0.5, posinf=1, neginf=0)
    for sn in stocksnames:
      stockstopresent=np.arange(10).astype(np.float32)
      k=k+1
      lo=lo+1
      stockssortedin = {};
      stockssortedout = {};
      mam = {}
      allthestocks=np.arange(35).astype(np.float32)
      allthetrends=np.arange(35).astype(np.float32)
      calcuall=np.arange(45).astype(np.float32)
      su1=np.arange(25).astype(np.float32)
      su2=np.arange(25).astype(np.float32)
      re1=np.arange(25).astype(np.float32)
      re2=np.arange(25).astype(np.float32)
      trendprediction=np.arange(25).astype(np.float32)
      momentumall=np.arange(35).astype(np.float32)
      momentum=np.arange(25).astype(np.float32)
      ppt=np.arange(25).astype(np.float32)
      inputnotorder = [];
      normalhelp = [];
      calcu=[]
      vol = [];
      stockname = sn
      inputnotorder = datadic[sn]
      r1 = len(inputnotorder)
      clear_output()
      display(lod)
      normalhelp = inputnotorder[r1-25:r1, 1:4].astype(np.float32)
      calcu=inputnotorder[r1-35:r1, 1:4].astype(np.float32)
      mini = np.min(normalhelp)
      maxi = np.max(normalhelp)
      mam[0] = [maxi, mini]
      normalhelp = (normalhelp - mini) / (maxi - mini)*2-1
      normalhelp = np.nan_to_num(normalhelp, nan=0.5, posinf=1, neginf=0)
      for q in range(0,25):
        ppt[q]=(calcu[q+9,0]+calcu[q+9,1]+calcu[q+9,2])/3
        su1[q]=((ppt[q]*2)-(calcu[q+9,0]))
        su2[q]=((ppt[q])+(calcu[q+9,1])-(calcu[q+9,0]))
        re1[q]=((ppt[q]*2)-(calcu[q+9,1]))
        re2[q]=((ppt[q])+(calcu[q+9,0])-(calcu[q+9,1]))
      for q in range(0,25):
        trendprediction[q]=((calcu[q+10,2] / calcu[q+9,2])-1)
      for q in range(0,25):
        momentum[q]=((calcu[q+10,2])/(calcu[q,2])-1)
 
      su1=(su1 - mini) / (maxi - mini)*2-1
      su2=(su2 - mini) / (maxi - mini)*2-1
      re1=(re1 - mini) / (maxi - mini)*2-1
      re2=(re2 - mini) / (maxi - mini)*2-1
      lod.value=lo
 
      
      normalhelp = np.column_stack((normalhelp, trendprediction))
      normalhelp = np.column_stack((normalhelp, momentum))
      normalhelp = np.column_stack((normalhelp, su1))
      normalhelp = np.column_stack((normalhelp, su2))
      normalhelp = np.column_stack((normalhelp, re1))
      normalhelp = np.column_stack((normalhelp, re2))
      normalhelp = np.column_stack((normalhelp, normalhelp1))
      stockssortedin[0] = normalhelp[0:25, :]
      x = list(stockssortedin.values())
      mm = list(mam.values())
      z1 = np.expand_dims(x[0], axis=0)
      z1=z1.astype(np.float32)
      p = model(z1)
      maxi, mini = mm[0]
      p = p.numpy()
      p=p.astype(np.float32)
      l = z1[0,:,2]
      l = ((l+1) * (maxi - mini)/2) + mini
 
 
      for q in range(0,10):
        stockstopresent[q]=(((l[24])*(p[0,q])))+(l[24])
      allthetrends=np.concatenate((trendprediction*100,p[0,:]*100), axis=None)
      allthestocks=np.concatenate((l,stockstopresent), axis=None)
      calcuall=np.concatenate((calcu[:,2],stockstopresent), axis=None)
      
      for q in range(0,35):
        momentumall[q]=((calcuall[q+10])/(calcuall[q]))-1
    
      trends[stocksnames[k]] = allthetrends
      predictionstocks[stocksnames[k]] = allthestocks
      su1alld[stocksnames[k]]=((su1+1) * (maxi - mini)/2) + mini
      su2alld[stocksnames[k]]=((su2+1) * (maxi - mini)/2) + mini
      re1alld[stocksnames[k]]=((re1+1) * (maxi - mini)/2) + mini
      re2alld[stocksnames[k]]=((re2+1) * (maxi - mini)/2) + mini
      momentumd[stocksnames[k]]=momentumall
      
    snf=stocksnames
    today = dt.date.today()+timedelta(1)
    nys = tc.get_calendar("NASDAQ")
    pas=list(nys.sessions_window(pd.Timestamp(nys.previous_close(pd.Timestamp(today)).strftime('%Y-%m-%d'), tz=pytz.UTC),-25).strftime('%m-%d'))
    fut=list(nys.sessions_window(pd.Timestamp(nys.previous_close(pd.Timestamp(today)).strftime('%Y-%m-%d'), tz=pytz.UTC),9).strftime('%m-%d'))
    dates=pas+fut[1:10]
    kiki=pd.DataFrame(trends,index=dates,columns=snf)
    kaka=pd.DataFrame(predictionstocks,index=dates,columns=snf)
    momentumdfd=pd.DataFrame(momentumd,index=dates,columns=snf)
    lnfig=len(snf)*2
    if lnfig<10:
      lnfig=10
 
    lod.close()
    clear_output()
    print('the number of stocks to invest: '+str(len(snf)))
    display(button5,button6,button7,button8,button10)
    
 
    kaka.to_csv('/content/prices.csv')
    kiki.to_csv('/content/precent change.csv')
    momentumdfd.to_csv('/content/momentum.csv')
    def foo6(a):
      plt.clf()
      plt.cla()
      plt.close()
      clear_output()
      axeses=kaka.plot(figsize=(25,lnfig),subplots=True,grid=True,sharex=False)
      z=0
      for s in snf:
        dfsu1all=pd.DataFrame(su1alld[s],index=dates[0:25],columns=[s+'support 1'])
        dfsu2all=pd.DataFrame(su2alld[s],index=dates[0:25],columns=[s+'support 2'])
        dfre1all=pd.DataFrame(re1alld[s],index=dates[0:25],columns=[s+'resist 1'])
        dfre2all=pd.DataFrame(re2alld[s],index=dates[0:25],columns=[s+'resist 2'])
        dfsu1all.plot(ax=axeses[z],grid=True,style=['o'])
        dfsu2all.plot(ax=axeses[z],grid=True,style=['o'])
        dfre1all.plot(ax=axeses[z],grid=True,style=['o'])
        dfre2all.plot(ax=axeses[z],grid=True,style=['o'])
        z=z+1
      plt.setp(axeses,xticks=np.arange(len(dates)),xticklabels=dates)
      display(button9)
    button6.on_click(foo6)
 
    def foo7(a):
      plt.clf()
      plt.cla()
      plt.close()      
      clear_output()
      kiki.plot(figsize=(25, lnfig), subplots=True, grid=True, sharex=False,kind='bar')
      display(button9)
    button7.on_click(foo7)
    def foo8(a):
      plt.clf()
      plt.cla()
      plt.close()      
      clear_output()
      axeses=momentumdfd.plot(figsize=(25,lnfig),subplots=True,grid=True,sharex=False)
      plt.setp(axeses,xticks=np.arange(len(dates)),xticklabels=dates)
      display(button9)  
    button8.on_click(foo8)
    def foo9(a):
      plt.clf()
      plt.cla()
      plt.close()
      clear_output()
      display(button5,button6,button7,button8,button10)
    button9.on_click(foo9)
    def foo10(a):
      files.download('/content/prices.csv') 
      files.download('/content/precent change.csv') 
      files.download('/content/momentum.csv') 
    button10.on_click(foo10)    
 
  def getdata(stocksnames):
    waiting=HTML('<marquee style="width:1000;heights=1000;color: red;" direction=right><b>connecting yhaoo</b></marquee>')
    display(waiting)
    datadic={}
    maxdate = dt.datetime.now()
    max1 = str(maxdate)
    max1 = max1.split(' ')
    max = max1[0]
    min = '2021-01-01'
    snp = yf.download('^GSPC', start=min, end=max,progress=False)  # 1927
    datadic['s&p'] = snp.to_numpy().astype(np.float32)
    for sn in stocksnames:
      stockname = sn
      stock = yf.Ticker(stockname)
      data = yf.download(stockname, start=min, end=max,progress=False)
      datadic[sn]=data.to_numpy().astype(np.float32)  
    return datadic
 
    
  def predictionbestof(siu,topstocks):
    clear_output()  
    dataformodel={}
    dicdata={}
    beststocks=[]
    np.seterr(divide='ignore', invalid='ignore')
    stocksnames=siu 
    dicdata=getdata(stocksnames)
    dataformodel=createmodeldata(stocksnames,dicdata,0)
    clear_output()
    beststocks=evaluatemodel(stocksnames,dataformodel,topstocks)
    if len(beststocks)<1:
      header=HTML('<div class="header"><h1>there is no success chance to none of the stocks.</h1></div>')
      display(header,button5)
    else:
      presentandsave(beststocks,dicdata)
 
 
  def prediction(siu):
    snf=[]
    np.seterr(divide='ignore', invalid='ignore')
    lo=0
    la=len(siu)
    lod=widgets.FloatProgress(value=lo,min=0,max=la,description='Loading:',bar_style='info',style={'bar_color': '#ffff00'},orientation='horizontal')
    clear_output()
    stocksnames=siu 
    predictionstocks={}
    trends={};
    su1alld={}
    su2alld={}
    re1alld={}
    re2alld={}
    momentumd={}
    k=-1
    countplus=0
    countminus=0
    maxdate = dt.datetime.now()
    max1 = str(maxdate)
    max1 = max1.split(' ')
    max = max1[0]
    min = '2021-01-01'
    snp = yf.download('^GSPC', start=min, end=max)  # 1927
    inputnotorder1 = snp.to_numpy()
    r2 = len(snp)
    normalhelp1 = inputnotorder1[r2-25:r2,3].astype(np.float32)
    mini1 = np.min(normalhelp1)
    maxi1 = np.max(normalhelp1)
    normalhelp1 = (normalhelp1 - mini1) / (maxi1 - mini1)*2-1
    normalhelp1 = np.nan_to_num(normalhelp1, nan=0.5, posinf=1, neginf=0)
    for sn in stocksnames:
      stockstopresent=np.arange(10).astype(np.float32)
      check1=0
      check2=0
      k=k+1
      lo=lo+1
      stockssortedin = {};
      stockssortedout = {};
      mam = {}
      allthestocks=np.arange(35).astype(np.float32)
      allthetrends=np.arange(35).astype(np.float32)
      calcuall=np.arange(45).astype(np.float32)
      su1=np.arange(25).astype(np.float32)
      su2=np.arange(25).astype(np.float32)
      re1=np.arange(25).astype(np.float32)
      re2=np.arange(25).astype(np.float32)
      trendprediction=np.arange(25).astype(np.float32)
      momentumall=np.arange(35).astype(np.float32)
      momentum=np.arange(25).astype(np.float32)
      ppt=np.arange(25).astype(np.float32)
      inputnotorder = [];
      normalhelp = [];
      calcu=[]
      vol = [];
      stockname = sn
      stock = yf.Ticker(stockname)
      data = yf.download(stockname, start=min, end=max)
      inputnotorder = data.to_numpy()
      r1 = len(data)
      clear_output()
      display(lod)
      normalhelp = inputnotorder[r1-25:r1, 1:4].astype(np.float32)
      calcu=inputnotorder[r1-35:r1, 1:4].astype(np.float32)
      mini = np.min(normalhelp)
      maxi = np.max(normalhelp)
      mam[0] = [maxi, mini]
      normalhelp = (normalhelp - mini) / (maxi - mini)*2-1
      normalhelp = np.nan_to_num(normalhelp, nan=0.5, posinf=1, neginf=0)
      for q in range(0,25):
        ppt[q]=(calcu[q+9,0]+calcu[q+9,1]+calcu[q+9,2])/3
        su1[q]=((ppt[q]*2)-(calcu[q+9,0]))
        su2[q]=((ppt[q])+(calcu[q+9,1])-(calcu[q+9,0]))
        re1[q]=((ppt[q]*2)-(calcu[q+9,1]))
        re2[q]=((ppt[q])+(calcu[q+9,0])-(calcu[q+9,1]))
      for q in range(0,25):
        trendprediction[q]=((calcu[q+10,2] / calcu[q+9,2])-1)
      for q in range(0,25):
        momentum[q]=((calcu[q+10,2])/(calcu[q,2])-1)
 
      su1=(su1 - mini) / (maxi - mini)*2-1
      su2=(su2 - mini) / (maxi - mini)*2-1
      re1=(re1 - mini) / (maxi - mini)*2-1
      re2=(re2 - mini) / (maxi - mini)*2-1
      lod.value=lo
 
      
      normalhelp = np.column_stack((normalhelp, trendprediction))
      normalhelp = np.column_stack((normalhelp, momentum))
      normalhelp = np.column_stack((normalhelp, su1))
      normalhelp = np.column_stack((normalhelp, su2))
      normalhelp = np.column_stack((normalhelp, re1))
      normalhelp = np.column_stack((normalhelp, re2))
      normalhelp = np.column_stack((normalhelp, normalhelp1))
      stockssortedin[0] = normalhelp[0:25, :]
      x = list(stockssortedin.values())
      mm = list(mam.values())
      z1 = np.expand_dims(x[0], axis=0)
      z1=z1.astype(np.float32)
      p = model(z1)
      maxi, mini = mm[0]
      p = p.numpy()
      p=p.astype(np.float32)
      l = z1[0,:,2]
      l = ((l+1) * (maxi - mini)/2) + mini
      for q in range(0,10):
        stockstopresent[q]=(((l[24])*(p[0,q])))+(l[24])
      allthetrends=np.concatenate((trendprediction*100,p[0,:]*100), axis=None)
      allthestocks=np.concatenate((l,stockstopresent), axis=None)
      calcuall=np.concatenate((calcu[:,2],stockstopresent), axis=None)
      
      for q in range(0,35):
        momentumall[q]=((calcuall[q+10])/(calcuall[q]))-1
 
      trends[stocksnames[k]] = allthetrends
      predictionstocks[stocksnames[k]] = allthestocks
      su1alld[stocksnames[k]]=((su1+1) * (maxi - mini)/2) + mini
      su2alld[stocksnames[k]]=((su2+1) * (maxi - mini)/2) + mini
      re1alld[stocksnames[k]]=((re1+1) * (maxi - mini)/2) + mini
      re2alld[stocksnames[k]]=((re2+1) * (maxi - mini)/2) + mini
      momentumd[stocksnames[k]]=momentumall
      snf.append(stocksnames[k])
      
 
      lod.close()
    clear_output()
    print('the number of stocks to invest: '+str(len(snf)))
    display(button5,button6,button7,button8,button10)
    today = dt.date.today()+timedelta(1)
    nys = tc.get_calendar("NASDAQ")
    pas=list(nys.sessions_window(pd.Timestamp(nys.previous_close(pd.Timestamp(today)).strftime('%Y-%m-%d'), tz=pytz.UTC),-25).strftime('%m-%d'))
    fut=list(nys.sessions_window(pd.Timestamp(nys.previous_close(pd.Timestamp(today)).strftime('%Y-%m-%d'), tz=pytz.UTC),9).strftime('%m-%d'))
    dates=pas+fut[1:10]
    yiyi=pd.DataFrame(trends,index=dates,columns=snf)
    yaya=pd.DataFrame(predictionstocks,index=dates,columns=snf)
    momentumdf=pd.DataFrame(momentumd,index=dates,columns=snf)
    lnfig=len(snf)*2
    if lnfig<10:
      lnfig=10
 
 
    yaya.to_csv('/content/prices.csv')
    yiyi.to_csv('/content/precent change.csv')
    momentumdf.to_csv('/content/momentum.csv')
    def foo6(a):
      plt.clf()
      plt.cla()
      plt.close()     
      clear_output() 
      axes=yaya.plot(figsize=(25,lnfig),subplots=True,grid=True,sharex=False)
      z=0
      for s in snf:
        dfsu1all=pd.DataFrame(su1alld[s],index=dates[0:25],columns=[s+'support 1'])
        dfsu2all=pd.DataFrame(su2alld[s],index=dates[0:25],columns=[s+'support 2'])
        dfre1all=pd.DataFrame(re1alld[s],index=dates[0:25],columns=[s+'resist 1'])
        dfre2all=pd.DataFrame(re2alld[s],index=dates[0:25],columns=[s+'resist 2'])
        dfsu1all.plot(ax=axes[z],grid=True,style=['o'])
        dfsu2all.plot(ax=axes[z],grid=True,style=['o'])
        dfre1all.plot(ax=axes[z],grid=True,style=['o'])
        dfre2all.plot(ax=axes[z],grid=True,style=['o'])
        z=z+1
      plt.setp(axes,xticks=np.arange(len(dates)),xticklabels=dates)
      display(button9)
    button6.on_click(foo6)
 
    def foo7(a): 
      plt.clf()
      plt.cla()
      plt.close()    
      clear_output()
      yiyi.plot(figsize=(25, lnfig), subplots=True, grid=True, sharex=False,kind='bar')
      display(button9)
    button7.on_click(foo7)
    def foo8(a):
      plt.clf()
      plt.cla()
      plt.close()
      clear_output()
      axes=momentumdf.plot(figsize=(25,lnfig),subplots=True,grid=True,sharex=False)
      plt.setp(axes,xticks=np.arange(len(dates)),xticklabels=dates)
      display(button9)  
    button8.on_click(foo8)
    def foo9(a):
      plt.clf()
      plt.cla()
      plt.close()
      clear_output()
      display(button5,button6,button7,button8,button10)
    button9.on_click(foo9)
    def foo10(a):
      files.download('/content/prices.csv') 
      files.download('/content/precent change.csv') 
      files.download('/content/momentum.csv') 
    button10.on_click(foo10)    
 
  header=HTML('<div class="header"><h1>choose the stocks you want to predict:</h1></div>')
  display(header)
  allthestocks=['AAPL','ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK',
            'BKNG', 'BLK', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX',
            'DD', 'DHR', 'DIS', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOGL', 'GS',
            'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ',
            'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'PEP', 'PFE',
            'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TSLA', 'TXN', 'UNH',
            'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM','ABT', 'ABMD', 'ATVI', 'AMD', 'AAP', 'AES', 'AFL',
            'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'LNT', 'GOOGL', 'GOOG', 'AMZN', 'AMCR', 'AEE',
            'AAL', 'AEP', 'AIG', 'AWK', 'AMP', 'ABC', 'AME', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BK', 'BAX', 'BDX','BBY', 'BIO', 'BLK', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR','CHRW', 'COG', 'CDNS', 'CZR', 'CPB', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF', 'SCHW', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'CTSH', 'CMCSA', 'CMA', 'CAG', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'CCI', 'CSX', 'CMI', 'DHI', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DRE', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ENPH', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EVRG', 'ES', 'RE', 'EXPE', 'EXPD', 'EXR', 'FFIV', 'FAST', 'FRT', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FLIR', 'FMC', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GNRC', 'GE', 'GIS', 'GPC', 'GL', 'GPN', 'GWW', 'HAL', 'HBI', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HFC', 'HOLX', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'INCY', 'IR', 'ICE', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'J', 'JBHT', 'SJM', 'JCI', 'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KLAC', 'KR', 'LB', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LEG', 'LDOS', 'LEN', 'LNC', 'LIN', 'LYV', 'LKQ', 'L', 'LUMN', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MKC', 'MXIM', 'MCK', 'MRK', 'MTD', 'MGM', 'MCHP', 'MU', 'MAA', 'MHK', 'TAP', 'MPWR', 'MNST', 'MCO', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NWL', 'NEM', 'NWSA', 'NWS', 'NLSN', 'NI', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NOV', 'NRG', 'NUE', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'OKE', 'OTIS', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PENN', 'PNR', 'PBCT', 'PKI', 'PRGO', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PGR', 'PLD', 'PRU', 'PTC', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'DGX', 'RL', 'RJF', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'SBAC', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SWKS', 'SNA', 'LUV', 'SWK', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TEL', 'TDY', 'TFX', 'TER', 'TXN', 'TXT', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TWTR', 'TYL', 'TSN', 'UDR', 'ULTA', 'UAA', 'UA', 'UAL', 'UPS', 'URI', 'UHS', 'UNM', 'VLO', 'VTR', 'VRSN', 'VRSK', 'VRTX', 'VFC', 'VIAC', 'VTRS', 'VNO', 'VMC', 'WRB', 'WAB', 'WBA', 'WM', 'WAT', 'WEC', 'WELL', 'WST', 'WDC', 'WU', 'WRK', 'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']
  allthestocks=sorted(allthestocks)
  stlist=widgets.SelectMultiple(options=allthestocks,description='')
  display(stlist)
  lay=widgets.Layout(width='auto',height='auto' )
  button = widgets.Button(description='predict with advise',layout=lay,disabled=False,button_style='success',tooltip='Click me',icon='check',)
  button2 = widgets.Button(description='predict without advise',layout=lay,disabled=False,button_style='danger',tooltip='Click me',icon='check')
  button3 = widgets.Button(description='choose all the stocks',layout=lay,disabled=False,button_style='primary',tooltip='Click me',icon='check')
  button4 = widgets.Button(description='done',disabled=False,button_style='success',tooltip='Click me',icon='check')
  button5 = widgets.Button(description='start over',disabled=False,button_style='success',tooltip='Click me',icon='check')
  button6 = widgets.Button(description='stocks price',disabled=False,button_style='success',tooltip='Click me',icon='check')
  button7 = widgets.Button(description='stocks trend',disabled=False,button_style='success',tooltip='Click me',icon='check')
  button8 = widgets.Button(description='stocks momentum',disabled=False,button_style='success',tooltip='Click me',icon='check')
  button9 = widgets.Button(description='back',disabled=False,button_style='danger',tooltip='Click me',icon='check')
  button10 = widgets.Button(description='download data',disabled=False,button_style='primary',tooltip='Click me',icon='check')
  display(button3,button,button2)
  def foo2(a):
    v=list(stlist.value)
    predictionbestof(v,amount.value)
  def foo(a):
    clear_output()
    v=list(stlist.value)
    global amount
    header=HTML('<div class="header"><h1>top __ stocks from my pick:</h1></div>')
    display(header)
    amount=widgets.BoundedIntText(min=0,max=len(v),step=1,description='',disabled=False)
    display(amount,button4)
    button4.on_click(foo2)
  button.on_click(foo)
  def regfoo(a):
    v=list(stlist.value)
    prediction(v)
  button2.on_click(regfoo)
  def choosefoo(a):
    stlist.value=allthestocks
  button3.on_click(choosefoo)
  def foo5(a):
    clear_output()
    display(stlist)
    display(button3,button,button2)
  button5.on_click(foo5)